In [1]:
import cugrad
a = cugrad.Tensor(2.0); a.label = 'a'
b = cugrad.Tensor(3.0); b.label = 'b'
c = cugrad.Tensor(1.0); c.label = 'c'
d = a * b; d.label = 'd'
e = d + c; e.label = 'e'
e.backward()

def dfs(t):
    print(t)
    for child in t.children:
        dfs(child)

dfs(e)

<Tensor data=7.000000, grad=1.000000, label=e>
<Tensor data=6.000000, grad=1.000000, label=d>
<Tensor data=2.000000, grad=0.000000, label=a>
<Tensor data=3.000000, grad=0.000000, label=b>
<Tensor data=1.000000, grad=1.000000, label=c>


In [7]:
import cugrad
from cugrad.nn import Layer

def draw_compute_graph(tensor, filename="compute_graph", format="png"):
    import graphviz
    import itertools

    dot = graphviz.Digraph(format=format)
    dot.attr(rankdir='LR', size='12,12')

    # Unique identifier generator
    uid = itertools.count()

    # Mapping from tensor id to node name to avoid duplicates
    tensor_id_to_node = {}
    op_id_to_node = {}

    # Define colors for different operations
    op_colors = {
        "Add": "lightblue",
        "Subtract": "lightgreen",
        "Multiply": "orange",
        "Divide": "pink",
        "Exp": "yellow",
        "Tanh": "purple",
        # Add more operations and colors as needed
    }

    def add_nodes(tensor):
        if id(tensor) in tensor_id_to_node:
            return

        current_id = next(uid)
        tensor_node = f"tensor_{current_id}"
        tensor_id_to_node[id(tensor)] = tensor_node

        # Node label includes label, data, grad
        label = f"{tensor.label}\nData: {tensor.data:.2f}\nGrad: {tensor.grad:.2f}"
        dot.node(tensor_node, label=label, shape='ellipse')

        if tensor.op:
            # Create an operation node
            op = tensor.op
            op_node = f"op_{next(uid)}"
            op_id_to_node[id(op)] = op_node
            op_label = op.op_type
            color = op_colors.get(op_label, "gray")  # Default color if op_type not found
            dot.node(op_node, label=op_label, shape='box', style='filled', fillcolor=color)

            # Connect operation to tensor
            dot.edge(op_node, tensor_node)

            # Connect child tensors to operation
            for child in tensor.children:
                add_nodes(child)
                child_node = tensor_id_to_node[id(child)]
                dot.edge(child_node, op_node)

    add_nodes(tensor)
    dot.render(filename, view=False)
    print(f"Computation graph saved as {filename}.{format}")

def main():
    # Create tensors
    a = cugrad.tensor.Tensor(2.0)
    a.label = 'a'
    b = cugrad.tensor.Tensor(3.0)
    b.label = 'b'
    c = cugrad.tensor.Tensor(1.0)
    c.label = 'c'
    
    # Perform operations
    d = a * b
    d.label = 'd'
    e = d + c
    e.label = 'e'
    
    # Perform backward pass
    e.backward()
    
    # Print tensors
    print(e)
    print(d)
    print(a)
    print(b)
    print(c)
    
    # Visualize the computation graph
    draw_compute_graph(e, filename="example_compute_graph", format="png")
    
    # Example usage of Layer
    x0 = cugrad.tensor.Tensor(2.0)
    x0.label = 'x0'
    x1 = cugrad.tensor.Tensor(3.0)
    x1.label = 'x1'
    x = [x0, x1]
    n = Layer(2, 3)
    
    res = n(x)
    for idx, t in enumerate(res):
        t.label = f"res_{idx}"
        print(t)
    
    # Perform backward pass on one of the results
    res[-1].backward()
    
    # Visualize the computation graph for the last result
    draw_compute_graph(res[-1], filename="layer_compute_graph", format="png")

if __name__ == "__main__":
    main()


<Tensor data=7.000000, grad=1.000000, label=e>
<Tensor data=6.000000, grad=1.000000, label=d>
<Tensor data=2.000000, grad=3.000000, label=a>
<Tensor data=3.000000, grad=2.000000, label=b>
<Tensor data=1.000000, grad=1.000000, label=c>
Computation graph saved as example_compute_graph.png
<Tensor data=0.390494, grad=0.000000, label=res_0>
<Tensor data=0.998506, grad=0.000000, label=res_1>
<Tensor data=0.810962, grad=0.000000, label=res_2>
Computation graph saved as layer_compute_graph.png
